<font size="+3" color='#053c96'><h2><center> Modeling</h2></center></font>
<figure>
<center><img src ="https://www.krasamo.com/wp-content/uploads/0730Building-ML-Models.jpg" width = "750" height = '600' alt="Classical Punk"/>
<font size="0" color='#053c96'><h4><center> Photo Credit: krasamo</h4></center></font>

<font size="+2" color='#053c96'><b> Contributor</b></font>  
<font size="+0" ><b> Umar Kabir</b></font>  

<a id='table-of-contents'></a>
[Table of Contents](#table-of-contents)

- [Modeling](#modeling)
  * [Data Transformation](#data-transformation)
  * [Model Selection](#model-selection)
  * [Model Training](#model-training)
  * [Model Evaluation](#model-evaluation)
  * [Hyperparameter Tuning](#hyperparameter-tuning)
- [Results](#results)
  * [Analysis Results](#analysis-results)
  * [Model Performance](#model-performance)
  * [Feature Importance](#feature-importance)
  * [Implications](#implications)
- [Conclusion](#conclusion)
  * [Summary](#summary)
  * [Limitations](#limitations)
  * [Recommendations](#recommendations)
- [References](#references)

<a id='importing-dependencies'></a>
<font size="+2" color='#053c96'><b> 1. Importing Libraries</b></font>  
[back to top](#table-of-contents)

In [1]:
import sys
# Insert the parent path relative to this notebook so we can import from the src folder.
sys.path.insert(0, "..")

from src.dependencies import *
from src.functions import *

<a id='data'></a>
<font size="+2" color='#053c96'><b> 2. Data</b></font>  
[back to top](#table-of-contents)  

The dataset was used for the well-known paper in genre classification "Musical genre classification of audio signals" by G. Tzanetakis and P. Cook in IEEE Transactions on Audio and Speech Processing 2002.

In [61]:
df = pd.read_csv("../data/processed_data.csv")

In [62]:
# Shuffle the data
df = shuffle(df)

In [63]:
df.head()

,Unnamed: 0,Genre,Filename,Tempo,Complexity,Timbre,Spectral Energy,Temporal Energy,Beats Mean,Central Moments Mean,Zero Crossing Rate Mean,RMSE Mean,Spectral Contrast Mean,Spectral Roll-off Mean,MFCC Mean,Chroma Mean,Spectral Centroid Mean,Spectral Bandwidth Mean,Spectral Flatness Mean,Loudness Mean,Harmonicity Mean,Percussiveness Mean,Beats std,Central Moments std,Zero Crossing Rate std,RMSE std,Spectral Contrast std,Spectral Roll-off std,MFCC std,Chroma std,Spectral Centroid std,Spectral Bandwidth std,Spectral Flatness std,Loudness std,Harmonicity std,Percussiveness std,Beats var,Central Moments var,Zero Crossing Rate var,RMSE var,Spectral Contrast var,Spectral Roll-off var,MFCC var,Chroma var,Spectral Centroid var,Spectral Bandwidth var,Spectral Flatness var,Loudness var,Harmonicity var,Percussiveness var,Beats sum,Central Moments sum,Zero Crossing Rate sum,RMSE sum,Spectral Contrast sum,Spectral Roll-off sum,MFCC sum,Chroma sum,Spectral Centroid sum,Spectral Bandwidth sum,Spectral Flatness sum,Loudness sum,Harmonicity sum,Percussiveness sum,Beats max,Central Moments max,Zero Crossing Rate max,RMSE max,Spectral Contrast max,Spectral Roll-off max,MFCC max,Chroma max,Spectral Centroid max,Spectral Bandwidth max,Spectral Flatness max,Loudness max,Harmonicity max,Percussiveness max,Beats min,Central Moments min,Zero Crossing Rate min,RMSE min,Spectral Contrast min,Spectral Roll-off min,MFCC min,Chroma min,Spectral Centroid min,Spectral Bandwidth min,Spectral Flatness min,Loudness min,Harmonicity min,Percussiveness min,Beats zscore,Central Moments zscore,Zero Crossing Rate zscore,RMSE zscore,Spectral Contrast zscore,Spectral Roll-off zscore,MFCC zscore,Chroma zscore,Spectral Centroid zscore,Spectral Bandwidth zscore,Spectral Flatness zscore,Loudness zscore,Harmonicity zscore,Percussiveness zscore,Beats kurtosis,Central Moments kurtosis,Zero Crossing Rate kurtosis,RMSE kurtosis,Spectral Contrast kurtosis,Spectral Roll-off kurtosis,MFCC kurtosis,Chroma kurtosis,Spectral Centroid kurtosis,Spectral Bandwidth kurtosis,Spectral Flatness kurtosis,Loudness kurtosis,Harmonicity kurtosis,Percussiveness kurtosis,Beats skew,Central Moments skew,Zero Crossing Rate skew,RMSE skew,Spectral Contrast skew,Spectral Roll-off skew,MFCC skew,Chroma skew,Spectral Centroid skew,Spectral Bandwidth skew,Spectral Flatness skew,Loudness skew,Harmonicity skew,Percussiveness skew,Beats iqr,Central Moments iqr,Zero Crossing Rate iqr,RMSE iqr,Spectral Contrast iqr,Spectral Roll-off iqr,MFCC iqr,Chroma iqr,Spectral Centroid iqr,Spectral Bandwidth iqr,Spectral Flatness iqr,Loudness iqr,Harmonicity iqr,Percussiveness iqr
904,904,rock,rock.00004.wav,103.359375,2.572210e-07,21.528547,1.988146e+10,30041.7580,635.959184,0.012449,0.035807,0.151337,66512.227156,4389.184570,49787.181998,0.588,1832.258174,2228.523490,0.005341,-16.702490,0.073103,0.033637,354.140060,0.019297,0.023376,0.037688,0.0,1829.336821,0.0,0.0,678.522603,562.165603,0.004583,2.356688,0.063472,0.019548,125415.182007,0.000372,0.000546,0.001420,0.0,3.346473e+06,0.0,0.0,460392.923110,316030.165497,0.000021,5.553978,0.004029,0.000382,31162,0.049798,0.214844,0.908022,66512.227156,26335.107422,49787.181998,0.588,10993.549045,13371.140940,0.032048,-100.214942,0.438620,0.201821,1246,0.045394,0.070801,0.199791,66512.227156,6449.194336,49787.181998,0.588,2615.237712,2902.175746,0.012513,-13.988486,0.150880,0.061212,28,-0.002224,0.011719,0.091283,66512.227156,2390.185547,49787.181998,0.588,1062.972739,1459.183674,0.000155,-20.792168,0.003680,0.008741,-4.531523e-18,-1.110223e-16,3.700743e-17,-1.619075e-16,NaN,2.035409e-16,NaN,NaN,1.295260e-16,5.551115e-17,-1.480297e-16,-1.850372e-17,-9.251859e-17,5.551115e-17,-1.162440,-0.747342,-1.571024,-1.208414,NaN,-1.971780,NaN,NaN,-1.946043,-1.694768,-1.417779,-0.995510,-1.922960,-1.542422,-0.000111,1.060051,0.353655,-0.302854,NaN,0.006096,NaN,NaN,-0.000164,-0.138686,0.288015,-0.579044,0.039841,-0.009778,592.00,0.016875,0.040405,0.053589,0.0,3474.920654,0.0,0.0,1273.148141

In [64]:
X = df.drop(['Genre', 'Filename', 'Unnamed: 0', 'Spectral Contrast zscore', 'MFCC zscore',
            'Chroma zscore', 'Spectral Contrast kurtosis', 'MFCC kurtosis',
            'Chroma kurtosis', 'Spectral Contrast skew', 'MFCC skew',
            'Chroma skew',], axis=1)
y = df['Genre']

<a id='modeling'></a>

<font size="+2" color='#053c96'><b> 4. Modeling</b></font>  
[back to top](#table-of-contents)

<a id='data-transformation'></a>
<font size="+0" color='green'><b> Feature Selection</b></font>  
[back to top](#table-of-contents)
<a id='modeling'></a>

In [65]:
# Initialize the OneHotEncoder
encoder = LabelEncoder()

# Fit and transform the 'Genre' column
y_encoded = encoder.fit_transform(y)

In [131]:
# Apply Min-Max scaling to make the features non-negative
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Initialize the feature selector
selector = SelectKBest(score_func=chi2, k=50)  # Select top 10 features based on chi-squared test

# Apply feature selection
X_selected = selector.fit_transform(X_scaled, y)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_features = X.columns[selected_indices]

In [132]:
# Create a logistic regression classifier
classifier = LogisticRegression()

# Initialize the RFE selector
rfe_selector = RFE(estimator=classifier, n_features_to_select=50)  # Select top 5 features

# Apply feature selection
X_selected = rfe_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_indices = rfe_selector.get_support(indices=True)

# Get the names of the selected features
selected_features1 = X.columns[selected_indices]


In [133]:
# Create a logistic regression classifier with L1 regularization (Lasso)
classifier = LogisticRegression(penalty='l1', solver='liblinear')

# Initialize the feature selector
selector = SelectFromModel(estimator=classifier, max_features=50)

# Apply feature selection
X_selected = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_features2 = X.columns[selected_indices]

In [134]:
# Perform feature selection using mutual information
selector = SelectKBest(score_func=mutual_info_classif, k=50)  # Select top 5 features based on mutual information

# Apply feature selection
X_selected = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_features3 = X.columns[selected_indices]


In [135]:
features_ = list(set(selected_features) | set(selected_features1) | set(selected_features2) | set(selected_features3))
print(features_)

['Loudness max', 'Spectral Bandwidth sum', 'Temporal Energy', 'Beats sum', 'Spectral Bandwidth iqr', 'Spectral Bandwidth skew', 'Beats skew', 'Spectral Roll-off skew', 'Central Moments var', 'Zero Crossing Rate sum', 'Beats std', 'Spectral Roll-off std', 'Spectral Contrast sum', 'Harmonicity sum', 'Harmonicity std', 'Central Moments kurtosis', 'MFCC sum', 'Spectral Flatness iqr', 'Spectral Roll-off var', 'MFCC Mean', 'Spectral Bandwidth std', 'Zero Crossing Rate kurtosis', 'Central Moments skew', 'Chroma max', 'RMSE var', 'Zero Crossing Rate min', 'Chroma min', 'Tempo', 'Chroma Mean', 'Central Moments iqr', 'Spectral Centroid sum', 'Loudness kurtosis', 'Harmonicity kurtosis', 'Spectral Flatness var', 'Loudness std', 'Spectral Contrast min', 'Spectral Roll-off iqr', 'RMSE iqr', 'Spectral Centroid Mean', 'Spectral Centroid max', 'Harmonicity skew', 'Spectral Flatness max', 'Beats Mean', 'Spectral Roll-off min', 'Spectral Contrast Mean', 'Spectral Flatness min', 'Percussiveness skew', 'RM

<a id='data-transformation'></a>
<font size="+0" color='green'><b> Data Transformation</b></font>  
[back to top](#table-of-contents)
<a id='modeling'></a>

In [136]:
# Calculate the mean and standard deviation for each column
mean = X.mean()
std = X.std()

# Perform Z-score normalization on the entire dataframe
train_df = (X - mean) / std
train_df = train_df.fillna(0)

In [137]:
# Applying natural logarithm transformation to all columns
train_df_transformed = X.apply(np.log)
# Replace all NaN values with 0
train_df_transformed = train_df_transformed.fillna(0)
# Replace all inf values with 0
train_df_transformed = train_df_transformed.replace([np.inf, -np.inf], 0)

In [138]:
# Perform standardization on the train dataframe
scaler = StandardScaler()
train_df_scaled = scaler.fit_transform(X)

# Create a PCA object
pca = PCA()

# Apply PCA on the scaled train dataframe
train_df_pca = pca.fit_transform(train_df_scaled)

# Create a new dataframe with the principal components
train_df_pca = pd.DataFrame(data=train_df_pca, columns=['PC{}'.format(i+1) for i in range(pca.n_components_)])

In [139]:
#Pareto Principle Split
X_train, X_test, y_train, y_test = train_test_split(train_df_transformed[features_], y_encoded, test_size=0.3, random_state=42)

<a id='model-selection'></a>

<font size="+1" color='#780404'><b> 4.1 Model Selection</b></font>  
[back to top](#table-of-contents)

In [154]:
# Multiclass Support Vector Machines (SVM)
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', OneVsRestClassifier(SVC(kernel='rbf', C=1, gamma='scale')))
])

# K-Means Clustering
kmeans_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('kmeans', KMeans(n_clusters=10))
])

# K-Nearest Neighbors (KNN)
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

# Feedforward Neural Network
cnn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', Sequential([
        Dense(64, activation='relu', input_shape=(len(features_),)),
        Dense(32, activation='relu'),
        Dense(10, activation='softmax')
    ]))
])
epochs = 50
batch_size = 32

# Multiclass Logistic Regression
logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])

# Multinomial Naive Bayes
nb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', MultinomialNB())
])

# Random Forest
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Gradient Boosting
gb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', GradientBoostingClassifier(n_estimators=100, random_state=42))
])

# Linear Support Vector Machines (SVM)
linear_svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LinearSVC())
])

# AdaBoost Classifier
adaboost_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', AdaBoostClassifier(n_estimators=100, random_state=42))
])

# XGBoost Classifier
xgb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', XGBClassifier(n_estimators=100, random_state=42))
])

# Multi-layer Perceptron (MLP) Classifier
mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42))
])

# Gaussian Process Classifier
gaussian_process_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', GaussianProcessClassifier(random_state=42))
])

# LightGBM Classifier
lgbm_pipeline = Pipeline([
    ('clf', LGBMClassifier(n_estimators=100, random_state=42))
])

# CatBoost Classifier
catboost_pipeline = Pipeline([
    ('clf', CatBoostClassifier(iterations=100, random_seed=42, verbose=0))
])

# Quadratic Discriminant Analysis (QDA)
qda_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', QuadraticDiscriminantAnalysis())
])

# Ridge Classifier
ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RidgeClassifier(alpha=1.0))
])

# Perceptron
perceptron_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', Perceptron(max_iter=1000, random_state=42))
])


<a id='model-training'></a>

<font size="+1" color='#780404'><b> 4.2 Model Training</b></font>  
[back to top](#table-of-contents)

In [157]:
# Fit SVM pipeline
svm_pipeline.fit(X_train, y_train)

# Fit KNN pipeline
knn_pipeline.fit(X_train, y_train)

# Fit Logistic Regression pipeline
logreg_pipeline.fit(X_train, y_train)

# Fit Multinomial Naive Bayes pipeline
#nb_pipeline.fit(X_train, y_train)

# Fit Random Forest pipeline
rf_pipeline.fit(X_train, y_train)

# Fit Gradient Boosting pipeline
gb_pipeline.fit(X_train, y_train)

# Fit Linear SVM pipeline
linear_svm_pipeline.fit(X_train, y_train)

# Fit AdaBoost pipeline
adaboost_pipeline.fit(X_train, y_train)

# Fit XGBoost pipeline
xgb_pipeline.fit(X_train, y_train)

# Fit Multi-layer Perceptron (MLP) pipeline
mlp_pipeline.fit(X_train, y_train)

# Fit Gaussian Process pipeline
gaussian_process_pipeline.fit(X_train, y_train)

# Fit LightGBM pipeline
lgbm_pipeline.fit(X_train, y_train)

# Fit CatBoost pipeline
catboost_pipeline.fit(X_train, y_train)

# Fit Quadratic Discriminant Analysis (QDA) pipeline
qda_pipeline.fit(X_train, y_train)

# Fit Ridge Classifier pipeline
ridge_pipeline.fit(X_train, y_train)

# Fit Perceptron pipeline
perceptron_pipeline.fit(X_train, y_train)

Epoch 1/50
18/18 [==============================] - 1s 19ms/step - loss: 1.1374 - accuracy: 0.6232 - val_loss: 2.2781 - val_accuracy: 0.2500
Epoch 2/50
18/18 [==============================] - 0s 3ms/step - loss: 1.0811 - accuracy: 0.6268 - val_loss: 2.4104 - val_accuracy: 0.2143
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 1.0770 - accuracy: 0.6339 - val_loss: 2.3618 - val_accuracy: 0.2429
Epoch 4/50
18/18 [==============================] - 0s 3ms/step - loss: 1.0521 - accuracy: 0.6500 - val_loss: 2.3834 - val_accuracy: 0.2500
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 1.0489 - accuracy: 0.6393 - val_loss: 2.3683 - val_accuracy: 0.2643
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 1.0468 - accuracy: 0.6464 - val_loss: 2.4521 - val_accuracy: 0.2357
Epoch 7/50
18/18 [==============================] - 0s 3ms/step - loss: 1.0289 - accuracy: 0.6661 - val_loss: 2.3646 - val_accuracy: 0.2286
Epoch 8/50
18/18 [=

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', Perceptron(random_state=42))])

<a id='model-evaluation'></a>

<font size="+1" color='#780404'><b> 4.3 Model Evaluation</b></font>  
[back to top](#table-of-contents)

In [159]:
# Initialize the lists for performance metrics and confusion matrices
f1_list = []
pr_list = []
rc_list = []
acc_list = []

# Evaluate additional models
# evaluate the SVM pipeline

y_pred_svm = svm_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_svm, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_svm, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_svm, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_svm))


# evaluate the KNN pipeline
y_pred_knn = knn_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_knn, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_knn, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_knn, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_knn))


# Evaluate Logistic Regression pipeline
y_pred_logreg = logreg_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_logreg, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_logreg, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_logreg, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_logreg))


# Evaluate Random Forest pipeline
y_pred_rf = rf_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_rf, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_rf, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_rf, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_rf))

# Evaluate Gradient Boosting pipeline
y_pred_gb = gb_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_gb, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_gb, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_gb, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_gb))

# Evaluate Linear SVM pipeline
y_pred_linear_svm = linear_svm_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_linear_svm, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_linear_svm, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_linear_svm, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_linear_svm))

# Evaluate AdaBoost pipeline
y_pred_adaboost = adaboost_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_adaboost, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_adaboost, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_adaboost, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_adaboost))

# Evaluate XGBoost pipeline
y_pred_xgb = xgb_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_xgb, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_xgb, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_xgb, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_xgb))

# Evaluate Multi-layer Perceptron (MLP) pipeline
y_pred_mlp = mlp_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_mlp, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_mlp, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_mlp, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_mlp))

# Evaluate Gaussian Process pipeline
y_pred_gaussian_process = gaussian_process_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_gaussian_process, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_gaussian_process, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_gaussian_process, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_gaussian_process))

# Evaluate LightGBM pipeline
y_pred_lgbm = lgbm_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_lgbm, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_lgbm, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_lgbm, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_lgbm))

# Evaluate CatBoost pipeline
y_pred_catboost = catboost_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_catboost, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_catboost, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_catboost, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_catboost))

# Evaluate Quadratic Discriminant Analysis (QDA) pipeline
y_pred_qda = qda_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_qda, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_qda, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_qda, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_qda))

# Evaluate Ridge Classifier pipeline
y_pred_ridge = ridge_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_ridge, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_ridge, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_ridge, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_ridge))

# Evaluate Perceptron pipeline
y_pred_perceptron = perceptron_pipeline.predict(X_test)
f1_list.append(f1_score(y_test, y_pred_perceptron, average='weighted'))
pr_list.append(precision_score(y_test, y_pred_perceptron, average='weighted'))
rc_list.append(recall_score(y_test, y_pred_perceptron, average='weighted'))
acc_list.append(accuracy_score(y_test, y_pred_perceptron))



In [160]:
# Create a list of model names
model_list = ['SVM', 'KNN', 'Logistic Regression', 'Random Forest', 'Gradient Boosting',
            'Linear SVM', 'AdaBoost', 'XGBoost', 'Multi-layer Perceptron', 'Gaussian Process',
            'LightGBM', 'CatBoost', 'QDA', 'Ridge Classifier', 'Perceptron']

In [161]:
result = pd.DataFrame({'Model': model_list, 'F1': f1_list, 'Precision': pr_list, 'Recall': rc_list, 'Accuracy': acc_list})

In [162]:
result

,Model,F1,Precision,Recall,Accuracy
0,SVM,0.404171,0.400079,0.430000,0.430000
1,KNN,0.314228,0.322350,0.320000,0.320000
2,Logistic Regression,0.393931,0.393724,0.396667,0.396667
3,Random Forest,0.425867,0.425307,0.433333,0.433333
4,Gradient Boosting,0.488082,0.491550,0.490000,0.490000
5,Linear SVM,0.386867,0.379972,0.400000,0.400000
6,AdaBoost,0.211073,0.246405,0.276667,0.276667
7,XGBoost,0.487744,0.495255,0.496667,0.496667
8,Multi-layer Perceptron,0.345478,0.356324,0.346667,0.346667
9,Gaussian Process,0.260263,0.293756,0.246667,0.246667


<a id='hyperparameter-tuning'></a>
<font size="+1" color='#780404'><b> 4.4 Hyperparameter Tuning</b></font>  
[back to top](#table-of-contents)

<a id='results'></a>
<font size="+2" color='#053c96'><b> 5. Results</b></font>  
[back to top](#table-of-contents)

<a id='analysis-results'></a>

<font size="+1" color='#780404'><b> 5.1 Analysis Results</b></font>  
[back to top](#table-of-contents)

<a id='model-performance'></a>

<font size="+1" color='#780404'><b> 5.2 Model Performance</b></font>  
[back to top](#table-of-contents)

<a id='feature-importance'></a>

<font size="+1" color='#780404'><b> 5.3 Feature Importance</b></font>  
[back to top](#table-of-contents)

<a id='implications'></a>

<font size="+1" color='#780404'><b> 5.4 Implications</b></font>  
[back to top](#table-of-contents)

<a id='conclusion'></a>

<font size="+2" color='#053c96'><b> 6. Conclusion</b></font>  
[back to top](#table-of-contents)

<a id='summary'></a>

<font size="+1" color='#780404'><b> 6.1 Summary</b></font>  
[back to top](#table-of-contents)

<a id='limitations'></a>

<font size="+1" color='#780404'><b> 6.2 Limitations</b></font>  
[back to top](#table-of-contents)

<a id='recommendations'></a>

<font size="+1" color='#780404'><b> 6.3 Recommendations</b></font>  
[back to top](#table-of-contents)

<a id='references'></a>

<font size="+2" color='#053c96'><b> 7. References</b></font>  
[back to top](#table-of-contents)